# 元々のjsonファイルを取得し，整形したものを出力

・ラベル名をtagsに統一

・ラベルとして使用されていた数値IDを文字型に変換

In [ ]:
#{"words": ["CHICAGO", "AT", "ATLANTA"], "ner": ["B-ORG", "O", "B-LOC"]}

In [5]:
import json

In [19]:
with open("data/original/id2coarse_tags.json","r") as f:
    label_coarse =json.loads(f.read())
    
with open("data/original/id2fine_tags.json","r") as f:
    label_fine =json.loads(f.read())

In [22]:
def coarse_str_tag(tags):
    new_tag = [label_coarse[str(i)] for i in tags]
    return new_tag
    
def fine_str_tag(tags):
    new_tag = [label_fine[str(i)] for i in tags]
    return new_tag

In [23]:
for filename in ["train","test","dev"]:
    
    data = []
    with open("data/original/"+filename+".json","r") as f:
        for line in f:
            data.append(json.loads(line))

    coarse = []
    for d in data:
        new_d = {}
        new_d["tokens"] = d["tokens"]
        new_d["tags"] = coarse_str_tag(d["coarse_tags"])
        coarse.append(new_d)
    with open("data/coarse/"+filename+".json","w") as f:
        for d in coarse:
            f.write(json.dumps(d))
            f.write("\n")

    fine = []
    for d in data:
        new_d = {}
        new_d["tokens"] = d["tokens"]
        new_d["tags"] = fine_str_tag(d["fine_tags"])
        fine.append(new_d)
    with open("data/fine/"+filename+".json","w") as f:
        for d in fine:
            f.write(json.dumps(d))
            f.write("\n")

In [7]:
label_coarse

[{'0': 'O',
  '1': 'art',
  '2': 'building',
  '3': 'event',
  '4': 'location',
  '5': 'organization',
  '6': 'other',
  '7': 'person',
  '8': 'product'}]

# BI-DA

・BIラベルのみに着目したデータ拡張

・交換率100％

In [1]:
import json
import random

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def SimpleDA_BI(dataname,filename,n):
    data = []
    with open("data/"+dataname+"/"+filename+".json","r") as f:
        for line in f:
            data.append(json.loads(line))
            
    alterna_labels = {}
    if dataname == "c":
        tag_json = "data/original/id2coarse_tags.json"
    else:
        tag_json = "data/original/id2fine_tags.json"
    with open(tag_json,"r") as f:
        label_coarse =json.loads(f.read())
        label_list = list(label_coarse.values())[1:]
    for label in label_list:
        alterna_labels[label] = set()
    for d in data:
        for token,tag in zip(d["tokens"],d["tags"]):
            if tag in label_list:
                alterna_labels[tag].add(token)

    data_f_path = "data/"+dataname+"_BI_x"+str(n)+"/"+filename+".json"
    memo_f_path = "log/"+dataname+"_BI_x"+str(n)+"_"+filename+"_memo.txt"
    
    
    with open(data_f_path,"w") as data_f, open(memo_f_path,"w") as memo_f:
        for d in data:
            data_f.write(json.dumps(d))
            data_f.write("\n")

        add_n = n*len(data)-len(data)
        print()
        print(dataname,filename,"add_n",add_n)
        
        for now in range(add_n):
            if now%10 == 0:
                print('\r%d / %d' %(now, add_n), end='')

            x = random.randint(0,len(data)-1)
            d = data[x]
            new_d = dict()
            new_d["tokens"] = []
            new_d["tags"] = data[x]["tags"]

            cnt = 0

            for token,tag in zip(d["tokens"],d["tags"]):
                if tag != "O":
                    if tag in alterna_labels:
                        token=random_choice_token(alterna_labels[tag]-{token})
                        cnt = cnt + 1
                    new_d["tokens"].append(token)

            data_f.write(json.dumps(new_d))
            data_f.write("\n")

            memo_f.write(str(x)+"\t"+" ".join(d["tokens"])+"\n")
            memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d["tokens"])+"\n")
            memo_f.write("\n")

        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [6]:
# for dataname in ["f"]:
#     for filename in ["dev","train"]:
#         SimpleDA_BI(dataname,filename,2)
#         SimpleDA_BI(dataname,filename,5)
#         SimpleDA_BI(dataname,filename,10)

# 形態素情報付与

In [4]:
import torch
device = torch.device("cuda:1")

In [5]:
from nltk.parse.stanford import *
import nltk
from nltk.tree import *
import pickle
import os

In [6]:
# import svgling

In [7]:
# !pip3 show svgling

In [8]:
java_path = "jre1.8.0_333/bin/java.exe"
os.environ['JAVAHOME'] = java_path

parser =  'stanford-corenlp/stanford-parser-full-2020-11-17/stanford-parser.jar'
parser_model = 'stanford-corenlp/stanford-corenlp-4.2.0-models-english.jar'

#POSタグの分析用
pos = StanfordParser(path_to_jar=parser, path_to_models_jar = parser_model)
#係り受け関係の分析用
dep_parser = StanfordDependencyParser(path_to_jar=parser, path_to_models_jar = parser_model)

def POSTagAnalysis(text):
    out = pos.raw_parse(text)
    out = list(out)
    tree = out[0]
    return tree

/tmp/ipykernel_3985206/2297462148.py:8: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  pos = StanfordParser(path_to_jar=parser, path_to_models_jar = parser_model)
/tmp/ipykernel_3985206/2297462148.py:10: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  dep_parser = StanfordDependencyParser(path_to_jar=parser, path_to_models_jar = parser_model)


In [48]:
import json

def TreePOS(filename):
    print("Momo")
    data = []
    with open("data/f/"+filename+".json","r") as f:
        for line in f:
            data.append(json.loads(line))
            
    new_document=[]
    document_trees = []
    new_sentence = set()
    error_sentence = set()
    c = 0
    
    for s in data:
        print('%d  /  %d' %(c, len(data)), end='\n')
        text = " ".join(s["tokens"])
        tree = POSTagAnalysis(text)
        document_trees.append(tree)
        new_d = dict()
        new_d["tokens"] = []
        new_d["tree_pos"] = []
        new_d["tags"] = []

        for i in range(len(tree.pos())):
            new_d["tokens"].append(tree.pos()[i][0])
            new_d["tree_pos"].append(tree.pos()[i][1])

        new_d["tags"]=s["tags"]
        new_document.append(new_d)
        c = c+1
        print(c)
        if c % 2 == 0 or c == len(data):
            print("hozon")
            with open("data/Stanford_coreNLP/"+filename+"_"+str(c)+"_tree.binaryfile", 'wb') as f:
                pickle.dump(document_trees,f)

            with open("data/Stanford_coreNLP/"+filename+"_"+str(c)+"_data.binaryfile", 'wb') as f:
                pickle.dump(new_document,f)

            with open("data/Stanford_coreNLP/"+filename+"_"+str(c)+"_error.binaryfile", 'wb') as f:
                pickle.dump(error_sentence,f)

In [49]:
for filename in ["dev","train"]:
    TreePOS(filename)

Momo
0  /  18823
1
1  /  18823
2
hozon
2  /  18823
3
3  /  18823
4
hozon
4  /  18823
5
5  /  18823
6
hozon
6  /  18823


KeyboardInterrupt: 

In [33]:
with open("data/Stanford_coreNLP/dev_3_tree.binaryfile", 'rb') as f:
    tree_data = pickle.load(f)

In [35]:
tree_data

3

In [17]:
print(len(new_document[0]["tokens"]))
print(len(new_document[0]["tree_pos"]))
print(len(new_document[0]["tags"]))

28
28
28


In [18]:
document_trees

[Tree('ROOT', [Tree('S', [Tree('NP', [Tree('NP', [Tree('DT', ['The']), Tree('JJ', ['final']), Tree('NN', ['stage'])]), Tree('PP', [Tree('IN', ['in']), Tree('NP', [Tree('NP', [Tree('DT', ['the']), Tree('NN', ['development'])]), Tree('PP', [Tree('IN', ['of']), Tree('NP', [Tree('DT', ['the']), Tree('NNP', ['Skyfox'])])])])])]), Tree('VP', [Tree('VBD', ['was']), Tree('NP', [Tree('NP', [Tree('DT', ['the']), Tree('NN', ['production'])]), Tree('PP', [Tree('IN', ['of']), Tree('NP', [Tree('NP', [Tree('DT', ['a']), Tree('NN', ['model'])]), Tree('PP', [Tree('IN', ['with']), Tree('NP', [Tree('NN', ['tricycle']), Tree('NN', ['landing']), Tree('NN', ['gear'])])])])]), Tree('S', [Tree('VP', [Tree('TO', ['to']), Tree('ADVP', [Tree('RBR', ['better'])]), Tree('VP', [Tree('VB', ['cater']), Tree('PP', [Tree('IN', ['for']), Tree('NP', [Tree('DT', ['the']), Tree('NN', ['pilot']), Tree('NN', ['training']), Tree('NN', ['market'])])])])])])])]), Tree('.', ['.'])])]),
 Tree('ROOT', [Tree('S', [Tree('NP', [Tree(